# Using `numba.jit` to speedup the computation of the Cityblock distance matrix 


In this notebook we implement a function to compute the Cityblock distance matrix using Numba's *just-in-time* compilation decorator. We compare it's performance to that of the corresponding non-decorated NumPy function.

In [ ]:
import numpy as np
import numba

In [ ]:
def cityblock_python(x, y):
    """Naive python implementation."""

    num_samples, num_feat = x.shape
    dist_matrix = np.empty((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in range(num_feat):
                r += np.abs(x[i][k] - y[j][k])
            dist_matrix[i][j] = r

    return dist_matrix


@numba.jit(nopython=True)
def cityblock_numba(x, y):
    """Implementation with numba."""

    num_samples, num_feat = x.shape
    dist_matrix = np.empty((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in range(num_feat):
                r += np.abs(x[i][k] - y[j][k])
            dist_matrix[i][j] = r

    return dist_matrix


def cityblock_broadcast(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = (x_ij - y_ij)^2
    """
    diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]

    return np.abs(diff).sum(axis=2)

In [ ]:
# Let's check that they all give the same result
a = 10. * np.random.random([100, 10])

print(np.abs(cityblock_python(a, a) - cityblock_numba(a, a)).max())
print(np.abs(cityblock_python(a, a) - cityblock_broadcast(a, a)).max())

In [ ]:
nsamples = 200
nfeat = 25

x = 10. * np.random.random([nsamples, nfeat])

%timeit cityblock_python(x,x)
%timeit cityblock_numba(x, x)

In [ ]:
nsamples = 2000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit cityblock_numba(x,x)
%timeit cityblock_broadcast(x, x)

## Conclusions

In cases where there's no possibility to do an implementation with NumPy vectorized operations, it's worth to give a try to Numba. It offers a significant improvement in performance compared to pure python, specially in situations where loops are unavoidable.